# **Data Wrangling**
En este notebook realizo las tareas identificadas en el **EDA**, entre ellas:
- Asignación de columnas que me interesan del INDEC (`Variedad`, `Unidad de medida` y `Octubre de 2025`)
- Limpieza de valores nulos
- Formateo a `snake_case` y eliminación de tildes
- Pasado de USD a pesos

## 1. Lectura de archivos _staging_ e importación de librerias
- Definición de la carpeta raíz e importación de librerias

In [ ]:
import sys, os

# Obtengo la ruta raíz del proyecto (subo un nivel desde /notebooks)
project_root = os.path.abspath("../")

# La agrego al path si no está
if project_root not in sys.path:
    sys.path.append(project_root)
    
# Importado de módulos y librerías
from scripts.utils import pasar_snake_case, aplicar_funcion_df, transformar_unidades, traductor_cols
import pandas as pd

- Lectura de archivos

In [ ]:
dir_staging = r'..\data\staging'

# Lectura de CSVs 
df_orders = pd.read_csv(dir_staging + r'\orders.csv')
df_items = pd.read_csv(dir_staging + r'\items_traducido.csv')
df_indec_temporal = pd.read_csv(dir_staging + r'\indec.csv')

- Asignación de columnas

In [ ]:
df_indec = pd.DataFrame()

# Asignación de columnas
df_indec['variedad'] = df_indec_temporal.iloc[:, 0]
df_indec['unidad_de_medida'] = df_indec_temporal.iloc[:, 1]
df_indec['precio'] = df_indec_temporal.iloc[:, 2]

# Revisión rápida
print(df_indec.isnull().sum().sum()) 

## 2. Limpieza de datos

- Eliminación de valores nulos

In [ ]:
df_items = df_items.dropna() # aunque ya son eliminados en "traducir_df.py"
df_indec = df_indec.dropna() 

## 3. Conversiones de datos

- Traducción de columnas del inglés al español

In [ ]:
# Traduzco las columnas
for df in [df_orders, df_items]:
    df.columns = [traductor_cols(col) for col in df.columns.tolist()]

> **Nota:** La traducción del DataFrame general es ejecutada en un script aparte (`traducir_df.py`). Para más información, acceder a él.

- Pasado a `snake_case` con una función auxiliar

In [ ]:
# Convertir valores a snake_case solo para ciertas columnas
aplicar_funcion_df(df_items, ["categoria", "subcategoria", "nombre_producto"], pasar_snake_case)
aplicar_funcion_df(df_indec, ["variedad", "unidad_de_medida", "precio"], pasar_snake_case) # la columna precio es una string, reemplazo "," y "."

- Conversión de precios y unidades de medida a kilogramos

In [ ]:
# Conversión a float
df_indec["precio"] = df_indec["precio"].astype(float)

# Utilizo función auxiliar que se encarga de pasar precios de diversas unidades/medidas a kg
df_indec = transformar_unidades(df_indec)

- Conversión de tipo de cambio (dólares a pesos)

In [ ]:

# Obtengo valor del dólar
with open(dir_staging + r"\dolar_blue.txt", "r") as db:
    valor_dolar = float(db.readline())

# Aplico cálculos
aplicar_funcion_df(df_items, ["precio", "costo_produccion"], lambda cant: round((cant * valor_dolar), 1))


## 5. Almacenado
- DataFrames en CSV

In [ ]:
clean_dir = r"..\data\clean"

# Almacenado de los DataFrames en formato CSV
df_orders.to_csv(clean_dir + r"\orders.csv", index=False)
df_items.to_csv(clean_dir + r"\items.csv", index=False)
df_indec.to_csv(clean_dir + r"\indec.csv", index=False)